In [201]:
name = "studio"
group = "roy"

# run a command line and save the output
def run(cmd):
    import subprocess
    print(cmd)
    result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # save result to a txt file in the current directory
    path = f"exif/{name}"
    with open(f"{path}.txt", "wb") as f:
        f.write(result.stdout)
        f.write(result.stderr)

cmd = f"exiftool {name}.jpg"
run(cmd)

# load the txt file as a dictionary 
path = f"exif/{name}.txt"
with open(path, 'r') as f:
    lightroom = False
    lines = f.readlines()
    d = {}
    for line in lines:
        if ':' in line:
            if "Dehaze" in line:
                lightroom = True
            key, value = line.split(':', 1)
            d[key] = value.strip()

if not lightroom:
    print("Not Lightroom compatible")

for key in d:
    d[key] = d[key].replace('\t', '')
    d[key] = d[key].replace('\n', '')
    d[key] = d[key].replace(' ', '')

# save as a json file
import json
path = f"exif/{name}.json"
with open(path, 'w') as f:
    json.dump(d, f, indent=4)


def convert_look(data):
    # check if data is empty
    if not data:
        return ""
    # remove spaces from the keys
    data = {key.strip().replace(" ", ""): value for key, value in data.items()}
    name = data["LookName"]
    amount = data["LookAmount"]
    group = "Adobe Raw"
    uuid = data["LookUUID"]
    supportsMonochrome = data["LookSupportsMonochrome"]

    result = f"""
 <crs:Look>
    <rdf:Description
     crs:Name="{name}"
     crs:Amount="{amount}"
     crs:Cluster="Adobe"
     crs:UUID="{uuid}"
     crs:SupportsMonochrome="{supportsMonochrome}"
     crs:Stubbed="true">
    <crs:Group>
     <rdf:Alt>
      <rdf:li xml:lang="x-default">{group}</rdf:li>
     </rdf:Alt>
    </crs:Group>
    </rdf:Description>
   </crs:Look>    
"""
    return result

def parse_tone_curve(data):
    
    def generate_seq(curve_name, curve_values):
        curve_name = curve_name.replace(" ", "")
        curve_name = curve_name.strip()
        # Split the string into a list of integers
        points = list(map(int, curve_values.split(',')))
        # Pair every two numbers into (x, y) tuples
        pairs = [(points[i], points[i + 1]) for i in range(0, len(points), 2)]
        # Generate the XML-like format
        seq = f"  <crs:{curve_name}>\n    <rdf:Seq>\n"
        for x, y in pairs:
            seq += f"     <rdf:li>{x}, {y}</rdf:li>\n"
        seq += "    </rdf:Seq>\n  </crs:{}>\n".format(curve_name.strip())
        return seq

    result = ""
    for key, value in data.items():
        curve_name = key.strip()
        result += generate_seq(curve_name, value)
    return result

    
# create tone curve map
def extract_tone_curve(data):
    tone_curves = {}
    for key, value in data.items():
        if 'ToneCurve' in key.replace(" ", ""):
            if "Name" not in key and "Look" not in key:
                tone_curves[key] = value
    return tone_curves

def extract_look(data):
    look_data = {}
    for key, value in data.items():
        if 'Look' in key.replace(" ", ""):
            if "Override" not in key:
                look_data[key] = value
    return look_data

import json
import os

# Function to remove all spaces from the keys of a dictionary
def remove_spaces_from_keys(data):
    return {key.replace(" ", ""): value for key, value in data.items()}

def create_lightroom_xmp_preset(exif_data, output_dir="presets"):
    exif_data = remove_spaces_from_keys(exif_data)
    os.makedirs(output_dir, exist_ok=True)
    preset_name = exif_data.get("FileName", "preset").split('.')[0] + ".xmp"
    
    def get_exif_value(key, default):
        value = exif_data.get(key, default)
        return value if value != "0" else default
    
    xmp_content = f"""
<x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="Adobe XMP Core 7.0-c000 1.000000, 0000/00/00-00:00:00        ">
 <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
  <rdf:Description rdf:about=""
    xmlns:crs="http://ns.adobe.com/camera-raw-settings/1.0/"
   crs:PresetType="Normal"
   crs:Cluster=""
   crs:UUID="79B8F47C758149D1B15FAEAFB97A5341"
   crs:SupportsAmount2="True"
   crs:SupportsAmount="True"
   crs:SupportsColor="True"
   crs:SupportsMonochrome="True"
   crs:SupportsHighDynamicRange="True"
   crs:SupportsNormalDynamicRange="True"
   crs:SupportsSceneReferred="True"
   crs:SupportsOutputReferred="True"
   crs:RequiresRGBTables="False"
   crs:CameraModelRestriction=""
   crs:Copyright=""
   crs:ContactInfo=""
   crs:Version="{get_exif_value('Version', '17.0')}"
   crs:ProcessVersion="{get_exif_value('ProcessVersion', '15.4')}"
   crs:Exposure2012="{get_exif_value('Exposure2012', '0.00')}"
   crs:Contrast2012="{get_exif_value('Contrast2012', '0')}"
   crs:Highlights2012="{get_exif_value('Highlights2012', '0')}"
   crs:Shadows2012="{get_exif_value('Shadows2012', '0')}"
   crs:Whites2012="{get_exif_value('Whites2012', '0')}"
   crs:Blacks2012="{get_exif_value('Blacks2012', '0')}"
   crs:Texture="{get_exif_value('Texture', '0')}"
   crs:Clarity2012="{get_exif_value('Clarity2012', '0')}"
   crs:Dehaze="{get_exif_value('Dehaze', '0')}"
   crs:Vibrance="{get_exif_value('Vibrance', '0')}"
   crs:Saturation="{get_exif_value('Saturation', '0')}"
   crs:ParametricShadows="{get_exif_value('ParametricShadows', '0')}"
   crs:ParametricDarks="{get_exif_value('ParametricDarks', '0')}"
   crs:ParametricLights="{get_exif_value('ParametricLights', '0')}"
   crs:ParametricHighlights="{get_exif_value('ParametricHighlights', '0')}"
   crs:ParametricShadowSplit="{get_exif_value('ParametricShadowSplit', '25')}"
   crs:ParametricMidtoneSplit="{get_exif_value('ParametricMidtoneSplit', '50')}"
   crs:ParametricHighlightSplit="{get_exif_value('ParametricHighlightSplit', '75')}"
   crs:Sharpness="{get_exif_value('Sharpness', '0')}"
   crs:LuminanceSmoothing="{get_exif_value('LuminanceSmoothing', '0')}"
   crs:ColorNoiseReduction="{get_exif_value('ColorNoiseReduction', '0')}"
   crs:HueAdjustmentRed="{get_exif_value('HueAdjustmentRed', '0')}"
   crs:HueAdjustmentOrange="{get_exif_value('HueAdjustmentOrange', '0')}"
   crs:HueAdjustmentYellow="{get_exif_value('HueAdjustmentYellow', '0')}"
   crs:HueAdjustmentGreen="{get_exif_value('HueAdjustmentGreen', '0')}"
   crs:HueAdjustmentAqua="{get_exif_value('HueAdjustmentAqua', '0')}"
   crs:HueAdjustmentBlue="{get_exif_value('HueAdjustmentBlue', '0')}"
   crs:HueAdjustmentPurple="{get_exif_value('HueAdjustmentPurple', '0')}"
   crs:HueAdjustmentMagenta="{get_exif_value('HueAdjustmentMagenta', '0')}"
   crs:SaturationAdjustmentRed="{get_exif_value('SaturationAdjustmentRed', '0')}"
   crs:SaturationAdjustmentOrange="{get_exif_value('SaturationAdjustmentOrange', '0')}"
   crs:SaturationAdjustmentYellow="{get_exif_value('SaturationAdjustmentYellow', '0')}"
   crs:SaturationAdjustmentGreen="{get_exif_value('SaturationAdjustmentGreen', '0')}"
   crs:SaturationAdjustmentAqua="{get_exif_value('SaturationAdjustmentAqua', '0')}"
   crs:SaturationAdjustmentBlue="{get_exif_value('SaturationAdjustmentBlue', '0')}"
   crs:SaturationAdjustmentPurple="{get_exif_value('SaturationAdjustmentPurple', '0')}"
   crs:SaturationAdjustmentMagenta="{get_exif_value('SaturationAdjustmentMagenta', '0')}"
   crs:LuminanceAdjustmentRed="{get_exif_value('LuminanceAdjustmentRed', '0')}"
   crs:LuminanceAdjustmentOrange="{get_exif_value('LuminanceAdjustmentOrange', '0')}"
   crs:LuminanceAdjustmentYellow="{get_exif_value('LuminanceAdjustmentYellow', '0')}"
   crs:LuminanceAdjustmentGreen="{get_exif_value('LuminanceAdjustmentGreen', '0')}"
   crs:LuminanceAdjustmentAqua="{get_exif_value('LuminanceAdjustmentAqua', '0')}"
   crs:LuminanceAdjustmentBlue="{get_exif_value('LuminanceAdjustmentBlue', '0')}"
   crs:LuminanceAdjustmentPurple="{get_exif_value('LuminanceAdjustmentPurple', '0')}"
   crs:LuminanceAdjustmentMagenta="{get_exif_value('LuminanceAdjustmentMagenta', '0')}"
   crs:SplitToningShadowHue="{get_exif_value('SplitToningShadowHue', '0')}"
   crs:SplitToningShadowSaturation="{get_exif_value('SplitToningShadowSaturation', '0')}"
   crs:SplitToningHighlightHue="{get_exif_value('SplitToningHighlightHue', '0')}"
   crs:SplitToningHighlightSaturation="{get_exif_value('SplitToningHighlightSaturation', '0')}"
   crs:SplitToningBalance="{get_exif_value('SplitToningBalance', '0')}"
   crs:ColorGradeMidtoneHue="{get_exif_value('ColorGradeMidtoneHue', '0')}"
   crs:ColorGradeMidtoneSat="{get_exif_value('ColorGradeMidtoneSat', '0')}"
   crs:ColorGradeShadowLum="{get_exif_value('ColorGradeShadowLum', '0')}"
   crs:ColorGradeMidtoneLum="{get_exif_value('ColorGradeMidtoneLum', '0')}"
   crs:ColorGradeHighlightLum="{get_exif_value('ColorGradeHighlightLum', '0')}"
   crs:ColorGradeBlending="{get_exif_value('ColorGradeBlending', '100')}"
   crs:ColorGradeGlobalHue="{get_exif_value('ColorGradeGlobalHue', '0')}"
   crs:ColorGradeGlobalSat="{get_exif_value('ColorGradeGlobalSat', '0')}"
   crs:ColorGradeGlobalLum="{get_exif_value('ColorGradeGlobalLum', '0')}"
   crs:AutoLateralCA="{get_exif_value('AutoLateralCA', '0')}"
   crs:LensProfileEnable="{get_exif_value('LensProfileEnable', '0')}"
   crs:DefringePurpleAmount="{get_exif_value('DefringePurpleAmount', '0')}"
   crs:DefringePurpleHueLo="{get_exif_value('DefringePurpleHueLo', '30')}"
   crs:DefringePurpleHueHi="{get_exif_value('DefringePurpleHueHi', '70')}"
   crs:DefringeGreenAmount="{get_exif_value('DefringeGreenAmount', '0')}"
   crs:DefringeGreenHueLo="{get_exif_value('DefringeGreenHueLo', '40')}"
   crs:DefringeGreenHueHi="{get_exif_value('DefringeGreenHueHi', '60')}"
   crs:GrainAmount="{get_exif_value('GrainAmount', '0')}"
   crs:PostCropVignetteAmount="{get_exif_value('PostCropVignetteAmount', '0')}"
   crs:ShadowTint="{get_exif_value('ShadowTint', '0')}"
   crs:RedHue="{get_exif_value('RedHue', '0')}"
   crs:RedSaturation="{get_exif_value('RedSaturation', '0')}"
   crs:GreenHue="{get_exif_value('GreenHue', '0')}"
   crs:GreenSaturation="{get_exif_value('GreenSaturation', '0')}"
   crs:BlueHue="{get_exif_value('BlueHue', '0')}"
   crs:BlueSaturation="{get_exif_value('BlueSaturation', '0')}"
   crs:HDREditMode="{get_exif_value('HDREditMode', '0')}"
   crs:CurveRefineSaturation="{get_exif_value('CurveRefineSaturation', '100')}"
   crs:OverrideLookVignette="{get_exif_value('OverrideLookVignette', 'False')}"
   crs:ToneCurveName2012="{get_exif_value('ToneCurveName2012', 'Linear')}"
   crs:HasSettings="{get_exif_value('HasSettings', 'True')}">
   <crs:Name>
    <rdf:Alt>
     <rdf:li xml:lang="x-default">{preset_name.split('.')[0]}</rdf:li>
    </rdf:Alt>
   </crs:Name>
   <crs:Group>
    <rdf:Alt>
     <rdf:li xml:lang="x-default">{group}</rdf:li>
    </rdf:Alt>
   </crs:Group>
   {parse_tone_curve(extract_tone_curve(exif_data))}
   <crs:Look>
    <rdf:Description
     crs:Name="Adobe Standard"
    <crs:Group>
     <rdf:Alt>
      <rdf:li xml:lang="x-default">Adobe Raw</rdf:li>
     </rdf:Alt>
    </crs:Group>
    {convert_look(extract_look(exif_data))}
    </rdf:Description>
   </crs:Look>
  </rdf:Description>
 </rdf:RDF>
</x:xmpmeta>
"""
    output_path = os.path.join(output_dir, preset_name)
    with open(output_path, 'w') as xmp_file:
        xmp_file.write(xmp_content)
    print(f"Preset saved to {output_path}")

if __name__ == "__main__":
    # load the json file as a dictionary
    path = f"exif/{name}.json"
    with open(path, 'r') as f:
        exif_json = f.read()
    exif_data = json.loads(exif_json)
    if lightroom:
        create_lightroom_xmp_preset(exif_data)
    else:
        print("Not Lightroom compatible")


exiftool studio.jpg
Preset saved to presets/studio.xmp


In [186]:
# # Input data
# look_data = {
#    "Look Amount                     ": "1.000000",
#     "Look Group                      ": "Profiles",
#     "Look Name                       ": "AdobeColor",
#     "Look Parameters Camera Profile  ": "AdobeStandard",
#     "Look Parameters Convert To Grayscale": "False",
#     "Look Parameters Look Table      ": "E1095149FDB39D7A057BAB208837E2E1",
#     "Look Parameters Process Version ": "11.0",
#     "Look Parameters Tone Curve PV2012": "0,0,22,16,40,35,127,127,224,230,240,246,255,255",
#     "Look Parameters Version         ": "13.3",
#     "Look Supports Amount            ": "false",
#     "Look Supports Monochrome        ": "false",
#     "Look Supports Output Referred   ": "false",
#     "Look UUID                       ": "B952C231111CD8E0ECCF14B86BAA7077"
# }

# # Convert and print the result
# xml_output = convert_look(look_data)
# print(xml_output)

# # Input data
# tone_curves = {
#     "Tone Curve PV2012               ": "0,28,50,43,101,100,168,184,255,231",
#     "Tone Curve PV2012 Blue          ": "0,0,65,65,130,133,196,193,255,255",
#     "Tone Curve PV2012 Green         ": "0,0,42,42,130,129,255,255",
#     "Tone Curve PV2012 Red           ": "0,0,51,53,132,129,208,205,255,255",
# }

# # Convert and print the result
# xml_result = parse_tone_curve(tone_curves)
# print(xml_result)


In [187]:
# # load the txt file as a dictionary 
# path = f"exif/{name}.txt"
# with open(path, 'r') as f:
#     lightroom = False
#     lines = f.readlines()
#     d = {}
#     for line in lines:
#         if ':' in line:
#             if "Dehaze" in line:
#                 lightroom = True
#             key, value = line.split(':', 1)
#             d[key] = value.strip()

# if not lightroom:
#     print("Not Lightroom compatible")

In [188]:
# for key in d:
#     d[key] = d[key].replace('\t', '')
#     d[key] = d[key].replace('\n', '')
#     d[key] = d[key].replace(' ', '')

# # save as a json file
# import json
# path = f"exif/{name}.json"
# with open(path, 'w') as f:
#     json.dump(d, f, indent=4)

    



In [189]:
# def convert_look(data):
#     # check if data is empty
#     if not data:
#         return ""
#     # remove spaces from the keys
#     data = {key.strip().replace(" ", ""): value for key, value in data.items()}
#     name = data["LookName"]
#     amount = data["LookAmount"]
#     group = "Adobe Raw"
#     uuid = data["LookUUID"]
#     supportsMonochrome = data["LookSupportsMonochrome"]

#     result = f"""
#  <crs:Look>
#     <rdf:Description
#      crs:Name="{name}"
#      crs:Amount="{amount}"
#      crs:Cluster="Adobe"
#      crs:UUID="{uuid}"
#      crs:SupportsMonochrome="{supportsMonochrome}"
#      crs:Stubbed="true">
#     <crs:Group>
#      <rdf:Alt>
#       <rdf:li xml:lang="x-default">{group}</rdf:li>
#      </rdf:Alt>
#     </crs:Group>
#     </rdf:Description>
#    </crs:Look>    
# """
#     return result


# # # Input data
# # look_data = {
# #    "Look Amount                     ": "1.000000",
# #     "Look Group                      ": "Profiles",
# #     "Look Name                       ": "AdobeColor",
# #     "Look Parameters Camera Profile  ": "AdobeStandard",
# #     "Look Parameters Convert To Grayscale": "False",
# #     "Look Parameters Look Table      ": "E1095149FDB39D7A057BAB208837E2E1",
# #     "Look Parameters Process Version ": "11.0",
# #     "Look Parameters Tone Curve PV2012": "0,0,22,16,40,35,127,127,224,230,240,246,255,255",
# #     "Look Parameters Version         ": "13.3",
# #     "Look Supports Amount            ": "false",
# #     "Look Supports Monochrome        ": "false",
# #     "Look Supports Output Referred   ": "false",
# #     "Look UUID                       ": "B952C231111CD8E0ECCF14B86BAA7077"
# # }

# # # Convert and print the result
# # xml_output = convert_look(look_data)
# # print(xml_output)


In [190]:
# def parse_tone_curve(data):
#     def generate_seq(curve_name, curve_values):
#         curve_name = curve_name.replace(" ", "")
#         curve_name = curve_name.strip()
#         # Split the string into a list of integers
#         points = list(map(int, curve_values.split(',')))
#         # Pair every two numbers into (x, y) tuples
#         pairs = [(points[i], points[i + 1]) for i in range(0, len(points), 2)]
#         # Generate the XML-like format
#         seq = f"  <crs:{curve_name}>\n    <rdf:Seq>\n"
#         for x, y in pairs:
#             seq += f"     <rdf:li>{x}, {y}</rdf:li>\n"
#         seq += "    </rdf:Seq>\n  </crs:{}>\n".format(curve_name.strip())
#         return seq

#     result = ""
#     for key, value in data.items():
#         curve_name = key.strip()
#         result += generate_seq(curve_name, value)
#     return result


# # # Input data
# # tone_curves = {
# #     "Tone Curve PV2012               ": "0,28,50,43,101,100,168,184,255,231",
# #     "Tone Curve PV2012 Blue          ": "0,0,65,65,130,133,196,193,255,255",
# #     "Tone Curve PV2012 Green         ": "0,0,42,42,130,129,255,255",
# #     "Tone Curve PV2012 Red           ": "0,0,51,53,132,129,208,205,255,255",
# # }

# # # Convert and print the result
# # xml_result = parse_tone_curve(tone_curves)
# # print(xml_result)

In [191]:
# # create tone curve map
# def extract_tone_curve(data):
#     tone_curves = {}
#     for key, value in data.items():
#         if 'ToneCurve' in key.replace(" ", ""):
#             if "Name" not in key and "Look" not in key:
#                 tone_curves[key] = value
#     return tone_curves

# def extract_look(data):
#     look_data = {}
#     for key, value in data.items():
#         if 'Look' in key.replace(" ", ""):
#             if "Override" not in key:
#                 look_data[key] = value
#     return look_data


In [192]:
# import json
# import os

# # Function to remove all spaces from the keys of a dictionary
# def remove_spaces_from_keys(data):
#     return {key.replace(" ", ""): value for key, value in data.items()}

# def create_lightroom_xmp_preset(exif_data, output_dir="presets"):
#     exif_data = remove_spaces_from_keys(exif_data)
#     os.makedirs(output_dir, exist_ok=True)
#     preset_name = exif_data.get("FileName", "preset").split('.')[0] + ".xmp"
    
#     def get_exif_value(key, default):
#         value = exif_data.get(key, default)
#         return value if value != "0" else default
    
#     xmp_content = f"""
# <x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="Adobe XMP Core 7.0-c000 1.000000, 0000/00/00-00:00:00        ">
#  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
#   <rdf:Description rdf:about=""
#     xmlns:crs="http://ns.adobe.com/camera-raw-settings/1.0/"
#    crs:PresetType="Normal"
#    crs:Cluster=""
#    crs:UUID="79B8F47C758149D1B15FAEAFB97A5341"
#    crs:SupportsAmount2="True"
#    crs:SupportsAmount="True"
#    crs:SupportsColor="True"
#    crs:SupportsMonochrome="True"
#    crs:SupportsHighDynamicRange="True"
#    crs:SupportsNormalDynamicRange="True"
#    crs:SupportsSceneReferred="True"
#    crs:SupportsOutputReferred="True"
#    crs:RequiresRGBTables="False"
#    crs:CameraModelRestriction=""
#    crs:Copyright=""
#    crs:ContactInfo=""
#    crs:Version="{get_exif_value('Version', '17.0')}"
#    crs:ProcessVersion="{get_exif_value('ProcessVersion', '15.4')}"
#    crs:Exposure2012="{get_exif_value('Exposure2012', '0.00')}"
#    crs:Contrast2012="{get_exif_value('Contrast2012', '0')}"
#    crs:Highlights2012="{get_exif_value('Highlights2012', '0')}"
#    crs:Shadows2012="{get_exif_value('Shadows2012', '0')}"
#    crs:Whites2012="{get_exif_value('Whites2012', '0')}"
#    crs:Blacks2012="{get_exif_value('Blacks2012', '0')}"
#    crs:Texture="{get_exif_value('Texture', '0')}"
#    crs:Clarity2012="{get_exif_value('Clarity2012', '0')}"
#    crs:Dehaze="{get_exif_value('Dehaze', '0')}"
#    crs:Vibrance="{get_exif_value('Vibrance', '0')}"
#    crs:Saturation="{get_exif_value('Saturation', '0')}"
#    crs:ParametricShadows="{get_exif_value('ParametricShadows', '0')}"
#    crs:ParametricDarks="{get_exif_value('ParametricDarks', '0')}"
#    crs:ParametricLights="{get_exif_value('ParametricLights', '0')}"
#    crs:ParametricHighlights="{get_exif_value('ParametricHighlights', '0')}"
#    crs:ParametricShadowSplit="{get_exif_value('ParametricShadowSplit', '25')}"
#    crs:ParametricMidtoneSplit="{get_exif_value('ParametricMidtoneSplit', '50')}"
#    crs:ParametricHighlightSplit="{get_exif_value('ParametricHighlightSplit', '75')}"
#    crs:Sharpness="{get_exif_value('Sharpness', '0')}"
#    crs:LuminanceSmoothing="{get_exif_value('LuminanceSmoothing', '0')}"
#    crs:ColorNoiseReduction="{get_exif_value('ColorNoiseReduction', '0')}"
#    crs:HueAdjustmentRed="{get_exif_value('HueAdjustmentRed', '0')}"
#    crs:HueAdjustmentOrange="{get_exif_value('HueAdjustmentOrange', '0')}"
#    crs:HueAdjustmentYellow="{get_exif_value('HueAdjustmentYellow', '0')}"
#    crs:HueAdjustmentGreen="{get_exif_value('HueAdjustmentGreen', '0')}"
#    crs:HueAdjustmentAqua="{get_exif_value('HueAdjustmentAqua', '0')}"
#    crs:HueAdjustmentBlue="{get_exif_value('HueAdjustmentBlue', '0')}"
#    crs:HueAdjustmentPurple="{get_exif_value('HueAdjustmentPurple', '0')}"
#    crs:HueAdjustmentMagenta="{get_exif_value('HueAdjustmentMagenta', '0')}"
#    crs:SaturationAdjustmentRed="{get_exif_value('SaturationAdjustmentRed', '0')}"
#    crs:SaturationAdjustmentOrange="{get_exif_value('SaturationAdjustmentOrange', '0')}"
#    crs:SaturationAdjustmentYellow="{get_exif_value('SaturationAdjustmentYellow', '0')}"
#    crs:SaturationAdjustmentGreen="{get_exif_value('SaturationAdjustmentGreen', '0')}"
#    crs:SaturationAdjustmentAqua="{get_exif_value('SaturationAdjustmentAqua', '0')}"
#    crs:SaturationAdjustmentBlue="{get_exif_value('SaturationAdjustmentBlue', '0')}"
#    crs:SaturationAdjustmentPurple="{get_exif_value('SaturationAdjustmentPurple', '0')}"
#    crs:SaturationAdjustmentMagenta="{get_exif_value('SaturationAdjustmentMagenta', '0')}"
#    crs:LuminanceAdjustmentRed="{get_exif_value('LuminanceAdjustmentRed', '0')}"
#    crs:LuminanceAdjustmentOrange="{get_exif_value('LuminanceAdjustmentOrange', '0')}"
#    crs:LuminanceAdjustmentYellow="{get_exif_value('LuminanceAdjustmentYellow', '0')}"
#    crs:LuminanceAdjustmentGreen="{get_exif_value('LuminanceAdjustmentGreen', '0')}"
#    crs:LuminanceAdjustmentAqua="{get_exif_value('LuminanceAdjustmentAqua', '0')}"
#    crs:LuminanceAdjustmentBlue="{get_exif_value('LuminanceAdjustmentBlue', '0')}"
#    crs:LuminanceAdjustmentPurple="{get_exif_value('LuminanceAdjustmentPurple', '0')}"
#    crs:LuminanceAdjustmentMagenta="{get_exif_value('LuminanceAdjustmentMagenta', '0')}"
#    crs:SplitToningShadowHue="{get_exif_value('SplitToningShadowHue', '0')}"
#    crs:SplitToningShadowSaturation="{get_exif_value('SplitToningShadowSaturation', '0')}"
#    crs:SplitToningHighlightHue="{get_exif_value('SplitToningHighlightHue', '0')}"
#    crs:SplitToningHighlightSaturation="{get_exif_value('SplitToningHighlightSaturation', '0')}"
#    crs:SplitToningBalance="{get_exif_value('SplitToningBalance', '0')}"
#    crs:ColorGradeMidtoneHue="{get_exif_value('ColorGradeMidtoneHue', '0')}"
#    crs:ColorGradeMidtoneSat="{get_exif_value('ColorGradeMidtoneSat', '0')}"
#    crs:ColorGradeShadowLum="{get_exif_value('ColorGradeShadowLum', '0')}"
#    crs:ColorGradeMidtoneLum="{get_exif_value('ColorGradeMidtoneLum', '0')}"
#    crs:ColorGradeHighlightLum="{get_exif_value('ColorGradeHighlightLum', '0')}"
#    crs:ColorGradeBlending="{get_exif_value('ColorGradeBlending', '100')}"
#    crs:ColorGradeGlobalHue="{get_exif_value('ColorGradeGlobalHue', '0')}"
#    crs:ColorGradeGlobalSat="{get_exif_value('ColorGradeGlobalSat', '0')}"
#    crs:ColorGradeGlobalLum="{get_exif_value('ColorGradeGlobalLum', '0')}"
#    crs:AutoLateralCA="{get_exif_value('AutoLateralCA', '0')}"
#    crs:LensProfileEnable="{get_exif_value('LensProfileEnable', '0')}"
#    crs:DefringePurpleAmount="{get_exif_value('DefringePurpleAmount', '0')}"
#    crs:DefringePurpleHueLo="{get_exif_value('DefringePurpleHueLo', '30')}"
#    crs:DefringePurpleHueHi="{get_exif_value('DefringePurpleHueHi', '70')}"
#    crs:DefringeGreenAmount="{get_exif_value('DefringeGreenAmount', '0')}"
#    crs:DefringeGreenHueLo="{get_exif_value('DefringeGreenHueLo', '40')}"
#    crs:DefringeGreenHueHi="{get_exif_value('DefringeGreenHueHi', '60')}"
#    crs:GrainAmount="{get_exif_value('GrainAmount', '0')}"
#    crs:PostCropVignetteAmount="{get_exif_value('PostCropVignetteAmount', '0')}"
#    crs:ShadowTint="{get_exif_value('ShadowTint', '0')}"
#    crs:RedHue="{get_exif_value('RedHue', '0')}"
#    crs:RedSaturation="{get_exif_value('RedSaturation', '0')}"
#    crs:GreenHue="{get_exif_value('GreenHue', '0')}"
#    crs:GreenSaturation="{get_exif_value('GreenSaturation', '0')}"
#    crs:BlueHue="{get_exif_value('BlueHue', '0')}"
#    crs:BlueSaturation="{get_exif_value('BlueSaturation', '0')}"
#    crs:HDREditMode="{get_exif_value('HDREditMode', '0')}"
#    crs:CurveRefineSaturation="{get_exif_value('CurveRefineSaturation', '100')}"
#    crs:OverrideLookVignette="{get_exif_value('OverrideLookVignette', 'False')}"
#    crs:ToneCurveName2012="{get_exif_value('ToneCurveName2012', 'Linear')}"
#    crs:HasSettings="{get_exif_value('HasSettings', 'True')}">
#    <crs:Name>
#     <rdf:Alt>
#      <rdf:li xml:lang="x-default">{preset_name.split('.')[0]}</rdf:li>
#     </rdf:Alt>
#    </crs:Name>
#    <crs:Group>
#     <rdf:Alt>
#      <rdf:li xml:lang="x-default">{group}</rdf:li>
#     </rdf:Alt>
#    </crs:Group>
#    {parse_tone_curve(extract_tone_curve(exif_data))}
#    <crs:Look>
#     <rdf:Description
#      crs:Name="Adobe Standard"
#     <crs:Group>
#      <rdf:Alt>
#       <rdf:li xml:lang="x-default">Adobe Raw</rdf:li>
#      </rdf:Alt>
#     </crs:Group>
#     {convert_look(extract_look(exif_data))}
#     </rdf:Description>
#    </crs:Look>
#   </rdf:Description>
#  </rdf:RDF>
# </x:xmpmeta>
# """
#     output_path = os.path.join(output_dir, preset_name)
#     with open(output_path, 'w') as xmp_file:
#         xmp_file.write(xmp_content)
#     print(f"Preset saved to {output_path}")

# if __name__ == "__main__":
#     # load the json file as a dictionary
#     path = f"exif/{name}.json"
#     with open(path, 'r') as f:
#         exif_json = f.read()
#     exif_data = json.loads(exif_json)
#     if lightroom:
#         create_lightroom_xmp_preset(exif_data)
#     else:
#         print("Not Lightroom compatible")